In [34]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.impute import KNNImputer
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

# Import the Data

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [4]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

In [6]:
df.head(5)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


# Searching for missing values

In [8]:
df.columns[df.isnull().sum()!=0]

Index(['lotfrontage', 'alley', 'masvnrtype', 'masvnrarea', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2',
       'electrical', 'fireplacequ', 'garagetype', 'garageyrblt',
       'garagefinish', 'garagequal', 'garagecond', 'poolqc', 'fence',
       'miscfeature'],
      dtype='object')

There are many variables with apparent null values.  Next I want to search for missing values that may not be apparent

In [25]:
nulls = df.columns[df.isnull().sum()!=0]
cols = df.select_dtypes('object')
for col in cols:
    if col not in nulls:
        print(col,": ",df[col].unique())

mszoning :  ['RL' 'RM' 'C (all)' 'FV' 'RH']
street :  ['Pave' 'Grvl']
lotshape :  ['Reg' 'IR1' 'IR2' 'IR3']
landcontour :  ['Lvl' 'Bnk' 'Low' 'HLS']
utilities :  ['AllPub' 'NoSeWa']
lotconfig :  ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
landslope :  ['Gtl' 'Mod' 'Sev']
neighborhood :  ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
condition1 :  ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
condition2 :  ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
bldgtype :  ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
housestyle :  ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
roofstyle :  ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
roofmatl :  ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
exterior1st :  [

now checking to see if any numeric values have a minimum as 0 as this could be a missing value

In [26]:
cols = df.select_dtypes(exclude='object')
for col in cols:
    if col not in nulls:
        print(col,": ",df[col].min())

id :  1
mssubclass :  20
lotarea :  1300
overallqual :  1
overallcond :  1
yearbuilt :  1872
yearremodadd :  1950
bsmtfinsf1 :  0
bsmtfinsf2 :  0
bsmtunfsf :  0
totalbsmtsf :  0
firstflrsf :  334
secondflrsf :  0
lowqualfinsf :  0
grlivarea :  334
bsmtfullbath :  0
bsmthalfbath :  0
fullbath :  0
halfbath :  0
bedroomabvgr :  0
kitchenabvgr :  0
totrmsabvgrd :  2
fireplaces :  0
garagecars :  0
garagearea :  0
wooddecksf :  0
openporchsf :  0
enclosedporch :  0
threessnporch :  0
screenporch :  0
poolarea :  0
miscval :  0
mosold :  1
yrsold :  2006
saleprice :  34900


As these have to do with counts and area, I'm assuming 0 is acceptable.  If I had more time I would look more into each

In [28]:
for null in nulls:
    print(null, ": ", df[null].isnull().sum())

lotfrontage :  259
alley :  1369
masvnrtype :  8
masvnrarea :  8
bsmtqual :  37
bsmtcond :  37
bsmtexposure :  38
bsmtfintype1 :  37
bsmtfintype2 :  38
electrical :  1
fireplacequ :  690
garagetype :  81
garageyrblt :  81
garagefinish :  81
garagequal :  81
garagecond :  81
poolqc :  1453
fence :  1179
miscfeature :  1406


In [33]:
(df.isnull().sum(axis=1).max())/df.shape[1]

0.18518518518518517

The greatest percentage of missing values in 1 row is about 19%.  If 20% of the data was missing, I would drop that row.

In [40]:
categorical = df.select_dtypes('object')

In [41]:
[x for x in nulls if x in categorical]

['alley',
 'masvnrtype',
 'bsmtqual',
 'bsmtcond',
 'bsmtexposure',
 'bsmtfintype1',
 'bsmtfintype2',
 'electrical',
 'fireplacequ',
 'garagetype',
 'garagefinish',
 'garagequal',
 'garagecond',
 'poolqc',
 'fence',
 'miscfeature']

Above are the categorical columns with nulls.  I want to one hot encode all categorical columns.  Then I will make sure the one hot encoded values are nans for these columns.  

In [46]:
df2 = pd.get_dummies(df,dummy_na=True)

In [51]:
for col in df2.columns:
    print(col)

id
mssubclass
lotfrontage
lotarea
overallqual
overallcond
yearbuilt
yearremodadd
masvnrarea
bsmtfinsf1
bsmtfinsf2
bsmtunfsf
totalbsmtsf
firstflrsf
secondflrsf
lowqualfinsf
grlivarea
bsmtfullbath
bsmthalfbath
fullbath
halfbath
bedroomabvgr
kitchenabvgr
totrmsabvgrd
fireplaces
garageyrblt
garagecars
garagearea
wooddecksf
openporchsf
enclosedporch
threessnporch
screenporch
poolarea
miscval
mosold
yrsold
saleprice
mszoning_C (all)
mszoning_FV
mszoning_RH
mszoning_RL
mszoning_RM
mszoning_nan
street_Grvl
street_Pave
street_nan
alley_Grvl
alley_Pave
alley_nan
lotshape_IR1
lotshape_IR2
lotshape_IR3
lotshape_Reg
lotshape_nan
landcontour_Bnk
landcontour_HLS
landcontour_Low
landcontour_Lvl
landcontour_nan
utilities_AllPub
utilities_NoSeWa
utilities_nan
lotconfig_Corner
lotconfig_CulDSac
lotconfig_FR2
lotconfig_FR3
lotconfig_Inside
lotconfig_nan
landslope_Gtl
landslope_Mod
landslope_Sev
landslope_nan
neighborhood_Blmngtn
neighborhood_Blueste
neighborhood_BrDale
neighborhood_BrkSide
neighborhood_Cl

In [ ]:
# next I need to drop those nan columns that did not have nan